In [1]:
import cloudknot as ck

In [10]:
import os

In [9]:
def comparison(subject):
    import os.path as op

    import boto3
    import numpy as np
    from AFQ import registration as reg
    import nibabel as nib
    from subprocess import call

    
    client = boto3.resource('s3')
    bucket_name = 'arokem.mri2mri'
    b = client.Bucket(bucket_name)
    ll = list(b.objects.all())
    real_a_files = []
    real_b_files = []
    fake_b_files = []

    for f in ll: 
        k = f.key.split('/')[-1]
        if k.startswith(subject):
            b.download_file(f.key, k)
            if k.endswith('real_A.npy'):
                real_a_files.append(k)
            elif k.endswith('real_B.npy'):
                real_b_files.append(k)
            elif k.endswith('fake_B.npy'):
                fake_b_files.append(k)
        if f.key.endswith("IITmean_V1.nii.gz"):
            b.download_file(f.key, "IITmean_V1.nii.gz")
        if f.key.endswith("IITmean_t1.nii.gz"):
            b.download_file(f.key, "IITmean_t1.nii.gz")
        if k.startswith(subject) and k.endswith("T1.nii.gz"):
            b.download_file(f.key, "T1.nii.gz")
        if k.startswith(subject) and k.endswith("DTI-00.nii.gz"):
            b.download_file(f.key, "DTI-00.nii.gz")
            

    real_a_files.sort()
    real_b_files.sort()
    fake_b_files.sort()
    real_a = np.zeros((len(real_a_files), 128, 128, 3))
    real_b = np.zeros((len(real_a_files), 128, 128, 3))
    fake_b = np.zeros((len(real_a_files), 128, 128, 3))
    for ii in range(len(real_a_files)):
        real_a[ii]= np.load(real_a_files[ii]) 
        real_b[ii]= np.load(real_b_files[ii]) 
        fake_b[ii]= np.load(fake_b_files[ii])
    
    meanV1_img = nib.load('IITmean_V1.nii.gz')
    meant1_img = nib.load('IITmean_t1.nii.gz')

    meanV1 = meanV1_img.get_data()
    meant1 = meant1_img.get_data()
    ants_call = ["antsBrainExtraction.sh", 
                 "-d", "3", "-a", "T1.nii.gz",
                 "-e" "T_template3.nii.gz", 
                 "-m", "T_template_BrainCerebellumProbabilityMask.nii.gz",
                 "-o", "T1_"]
    
    call(ants_call)            
    ants_brain_img = nib.load('T1_BrainExtractionBrain.nii.gz')
    ants_brain = ants_brain_img.get_data()
    transformed, affine = reg.affine_registration(meant1, ants_brain, meant1_img.affine, ants_brain_img.affine)   
    warped_meant1, mapping = reg.syn_registration(meant1, ants_brain, moving_affine=meant1_img.affine, static_affine=ants_brain_img.affine, prealign=affine)
    nib.save(nib.Nifti1Image(warped_meant1, ants_brain_img.affine), 'meant1_warped_to_ants_brain.nii.gz')
    mapped = np.concatenate([mapping.transform(meanV1[..., 0])[..., np.newaxis], 
                             mapping.transform(meanV1[..., 1])[..., np.newaxis], 
                             mapping.transform(meanV1[..., 2])[..., np.newaxis]], -1) 
    DWI_img = nib.load('DTI-00.nii.gz')
    DWI_affine = DWI_img.affine
    resamp = np.concatenate([reg.resample(mapped[..., 0], DWI_img.get_data(), ants_brain_img.affine, DWI_affine)[..., np.newaxis], 
                             reg.resample(mapped[..., 1], DWI_img.get_data(), ants_brain_img.affine, DWI_affine)[..., np.newaxis], 
                             reg.resample(mapped[..., 2], DWI_img.get_data(), ants_brain_img.affine, DWI_affine)[..., np.newaxis]], -1)[:,:,1:-1]
                         
    resamp = np.moveaxis(resamp, 2, 0)
    mask = (np.abs(real_b).sum(-1)) > 0 
    angle_fake = np.rad2deg(np.arccos(np.clip((real_b[mask] * fake_b[mask]).sum(axis=-1), -1, 1)))
    angle_fake = np.min(np.array([angle_fake, 180-angle_fake]), 0)
    angle_template = np.rad2deg(np.arccos(np.clip((real_b[mask] * resamp[mask]).sum(axis=-1), -1, 1)))
    angle_template = np.min(np.array([angle_template, 180-angle_template]), 0)
    
    

In [2]:
import boto3

In [3]:
client = boto3.resource('s3')
bucket_name = 'arokem.mri2mri'
b = client.Bucket(bucket_name)

In [4]:
ll = list(b.objects.all())

In [5]:
subject = "IXI029"

In [6]:
real_a_files = []
real_b_files = []
fake_b_files = []

for f in ll: 
    k = f.key
    if k.split('/')[1].startswith(subject):
        if k.endswith('real_A.npy'):
            real_a_files.append(k)
        elif k.endswith('real_B.npy'):
            real_b_files.append(k)
        elif k.endswith('fake_B.npy'):
            fake_b_files.append(k)


In [7]:
real_a_files.sort()

In [8]:
real_a_files

['t1_pdd_cosine_L1_unet128_T3_3d/IXI029-Guys_0001_real_A.npy',
 't1_pdd_cosine_L1_unet128_T3_3d/IXI029-Guys_0002_real_A.npy',
 't1_pdd_cosine_L1_unet128_T3_3d/IXI029-Guys_0004_real_A.npy',
 't1_pdd_cosine_L1_unet128_T3_3d/IXI029-Guys_0005_real_A.npy',
 't1_pdd_cosine_L1_unet128_T3_3d/IXI029-Guys_0007_real_A.npy',
 't1_pdd_cosine_L1_unet128_T3_3d/IXI029-Guys_0008_real_A.npy',
 't1_pdd_cosine_L1_unet128_T3_3d/IXI029-Guys_0009_real_A.npy',
 't1_pdd_cosine_L1_unet128_T3_3d/IXI029-Guys_0010_real_A.npy',
 't1_pdd_cosine_L1_unet128_T3_3d/IXI029-Guys_0011_real_A.npy',
 't1_pdd_cosine_L1_unet128_T3_3d/IXI029-Guys_0013_real_A.npy',
 't1_pdd_cosine_L1_unet128_T3_3d/IXI029-Guys_0014_real_A.npy',
 't1_pdd_cosine_L1_unet128_T3_3d/IXI029-Guys_0015_real_A.npy',
 't1_pdd_cosine_L1_unet128_T3_3d/IXI029-Guys_0016_real_A.npy',
 't1_pdd_cosine_L1_unet128_T3_3d/IXI029-Guys_0018_real_A.npy',
 't1_pdd_cosine_L1_unet128_T3_3d/IXI029-Guys_0019_real_A.npy',
 't1_pdd_cosine_L1_unet128_T3_3d/IXI029-Guys_0021_real_